### LSTM model for POS tagging

In [1]:
import torch

In [2]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super().__init__()
        self.hidden_data = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, hidden_dim)
        
        #emedding to hidden state
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        #hidden state to tag space
        self.hidden2tag = torch.Linear(hidden_dim, target_size)


    def forward(self, x):
        embed = self.word_embeddings(x)
        lstm_out, _ = self.lstm(embed.view(len(x), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        tag_scores = torch.nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores





Prepare Data

In [3]:
def prepare_sequence(seq, word_to_ix):

    idxs = [word_to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
                    ###Tags: 'DET': Determiner, 'NN': Noun, 'V': Verb
                    ('The dog ate the pie'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
                    ('All hail the king'.split(), ['DET', 'V', 'DET', 'NN']),

                ]

#Assign each word a unique index
word_to_ix={}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

#unique index for each tag
tag_to_ix = {'DET':0, 'NN':1, 'V':2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'pie': 4, 'All': 5, 'hail': 6, 'king': 7}


Train the model

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

loss_fn = torch.nn.NLLLoss()
optimizer = torch.nn.optim.SGD(lr=1e-2)


